In [ ]:
## This file implements neural networks for p11presabs_qual with four replicates.
## We compute the mean and standarad deviation of training and test accuracies.
## We also compute the mean and standard deviation of AUC ROC values for four replicates.

In [1]:
from numpy.random import seed
import numpy as np
seed(100)
import tensorflow
tensorflow.random.set_seed(123)

In [2]:
import pandas as pd

df = pd.read_csv('/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/p11presabs_qual.csv')
df.shape

(253, 953)

In [3]:
df.rename(columns={'Unnamed: 0':'id'}, inplace=True)

In [4]:
df['pheno']

0      2
1      1
2      2
3      2
4      2
      ..
248    2
249    1
250    2
251    2
252    2
Name: pheno, Length: 253, dtype: int64

In [5]:
df.head()

,id,TTTTTTGTAATTTT,TTTTTTGTAATTTTT,TTTTTTATTTTGGAT,TTTTTTATTTTGGATAA,TTTTTTATTTTGGATAAAAGGAG,TTTTTTAGTCGTTTTT,TTTTTATCGTTTACT,TTTTTAGTCGTTTTT,TTTTTAGTCGTTTTTT,...,group_6449,group_8588,group_10026,group_10028,group_10211,group_2860,group_5746,group_7042,group_867,pheno
0,107,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2
1,109,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,1
2,115,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,2
3,120335,1,1,1,1,1,1,1,1,1,...,0,1,0,0,0,0,0,0,0,2
4,120337,1,1,1,1,1,1,1,1,1,...,0,1,0,0,0,0,0,0,0,2


In [6]:
df['pheno'].value_counts()

2    181
1     47
0     25
Name: pheno, dtype: int64

In [7]:
df_clean = df.drop(columns=['id'])

In [8]:
df_clean.shape

(253, 952)

In [9]:
df_clean.head()

,TTTTTTGTAATTTT,TTTTTTGTAATTTTT,TTTTTTATTTTGGAT,TTTTTTATTTTGGATAA,TTTTTTATTTTGGATAAAAGGAG,TTTTTTAGTCGTTTTT,TTTTTATCGTTTACT,TTTTTAGTCGTTTTT,TTTTTAGTCGTTTTTT,TTTTTAGGTAAGG,...,group_6449,group_8588,group_10026,group_10028,group_10211,group_2860,group_5746,group_7042,group_867,pheno
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2
1,1,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,1
2,1,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,2
3,1,1,1,1,1,1,1,1,1,1,...,0,1,0,0,0,0,0,0,0,2
4,1,1,1,1,1,1,1,1,1,1,...,0,1,0,0,0,0,0,0,0,2


In [10]:
X = df.loc[:, df.columns != 'pheno']
y = df['pheno']
print(X.shape, y.shape)

(253, 952) (253,)


In [11]:
# over-sampling
from collections import Counter
from sklearn.datasets import make_classification
from imblearn.over_sampling import RandomOverSampler
overS = RandomOverSampler(random_state=100)
X_over, y_over = overS.fit_resample(X, y)
print(sorted(Counter(y_over).items()))

/Users/Rebecca/anaconda3/lib/python3.7/site-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)
/Users/Rebecca/anaconda3/lib/python3.7/site-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.ensemble.bagging module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.ensemble. Anything that cannot be imported from sklearn.ensemble is now part of the private API.
  warnings.warn(message, FutureWarning)
/Users/Rebecca/anaconda3/lib/python3.7/site-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.ensemble.base module is  deprecated in versi

[(0, 181), (1, 181), (2, 181)]


/Users/Rebecca/anaconda3/lib/python3.7/site-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.utils.testing module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.utils. Anything that cannot be imported from sklearn.utils is now part of the private API.
  warnings.warn(message, FutureWarning)
/Users/Rebecca/anaconda3/lib/python3.7/site-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.metrics.classification module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.metrics. Anything that cannot be imported from sklearn.metrics is now part of the private API.
  warnings.warn(message, FutureWarning)
/Users/Rebecca/anaconda3/lib/python3.7/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is depreca

In [12]:
############# Fully-Connected Neural Network ################

In [13]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.regularizers import l1

In [14]:
# split into train, test data (over)
from sklearn.model_selection import train_test_split
X_train_over, X_test_over, y_train_over, y_test_over = train_test_split(X_over, y_over,
                                                    test_size = 0.3,
                                                    random_state=123,
                                                    stratify=y_over)

In [15]:
dat = pd.DataFrame(X_test_over[:,0])
dat['test'] = y_test_over

In [16]:
dat

,0,test
0,SR1129,2
1,NRS185,2
2,NRS243,1
3,BCH-SA-04,0
4,504,1
...,...,...
158,CFBREBSa131,2
159,CFBREBSa133,1
160,NRS256,2
161,GA48963,1


In [17]:
X_train_over = X_train_over[:,1:]
X_test_over = X_test_over[:,1:]

In [18]:
#### neural network on over-sampling data
model1_over = Sequential([
    Dense(32, activation='relu', input_shape=(X_train_over.shape[1],)),
    Dense(3, activation='softmax'),
])

In [19]:
model1_over.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [20]:
model1_over.fit(X_train_over, y_train_over,
          batch_size=32, epochs=100,
          validation_data=(X_test_over, y_test_over))

Train on 380 samples, validate on 163 samples
Epoch 1/100
380/380 [==============================] - 0s 490us/step - loss: 1.2953 - accuracy: 0.3947 - val_loss: 1.1719 - val_accuracy: 0.4724
Epoch 2/100
380/380 [==============================] - 0s 236us/step - loss: 0.9334 - accuracy: 0.5342 - val_loss: 0.9308 - val_accuracy: 0.6258
Epoch 3/100
380/380 [==============================] - 0s 181us/step - loss: 0.8581 - accuracy: 0.6158 - val_loss: 0.8329 - val_accuracy: 0.6196
Epoch 4/100
380/380 [==============================] - 0s 176us/step - loss: 0.7640 - accuracy: 0.6737 - val_loss: 0.7840 - val_accuracy: 0.6196
Epoch 5/100
380/380 [==============================] - 0s 196us/step - loss: 0.6978 - accuracy: 0.6895 - val_loss: 0.7918 - val_accuracy: 0.6503
Epoch 6/100
380/380 [==============================] - 0s 184us/step - loss: 0.6610 - accuracy: 0.7079 - val_loss: 0.8039 - val_accuracy: 0.6564
Epoch 7/100
380/380 [==============================] - 0s 250us/step - loss: 0.6101 

Epoch 57/100
380/380 [==============================] - 0s 198us/step - loss: 0.1616 - accuracy: 0.9395 - val_loss: 0.3977 - val_accuracy: 0.8773
Epoch 58/100
380/380 [==============================] - 0s 266us/step - loss: 0.1585 - accuracy: 0.9474 - val_loss: 0.3943 - val_accuracy: 0.8834
Epoch 59/100
380/380 [==============================] - 0s 246us/step - loss: 0.1472 - accuracy: 0.9526 - val_loss: 0.4080 - val_accuracy: 0.8896
Epoch 60/100
380/380 [==============================] - 0s 291us/step - loss: 0.1628 - accuracy: 0.9553 - val_loss: 0.4446 - val_accuracy: 0.8282
Epoch 61/100
380/380 [==============================] - 0s 156us/step - loss: 0.1808 - accuracy: 0.9316 - val_loss: 0.4160 - val_accuracy: 0.8712
Epoch 62/100
380/380 [==============================] - 0s 277us/step - loss: 0.1743 - accuracy: 0.9447 - val_loss: 0.4479 - val_accuracy: 0.8466
Epoch 63/100
380/380 [==============================] - 0s 269us/step - loss: 0.2284 - accuracy: 0.9184 - val_loss: 0.4941 -

In [37]:
acc_test_over = model1_over.evaluate(X_test_over, y_test_over)[1]
print('over-sampling test accuracy: %.2f%%' % (acc_test_over*100))

163/163 [==============================] - 0s 111us/step
over-sampling test accuracy: 87.12%


In [21]:
pred = model1_over.predict_classes(X_test_over)
pred

array([0, 2, 1, 0, 1, 0, 0, 0, 1, 1, 2, 0, 0, 2, 0, 0, 2, 2, 0, 1, 1, 0,
       0, 2, 0, 0, 1, 1, 0, 0, 1, 2, 1, 0, 1, 0, 1, 1, 1, 1, 0, 2, 0, 2,
       1, 1, 2, 1, 0, 0, 0, 2, 1, 1, 1, 2, 1, 2, 0, 2, 0, 1, 0, 1, 1, 0,
       0, 0, 1, 2, 0, 1, 0, 1, 2, 1, 1, 2, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1,
       1, 2, 0, 1, 1, 0, 1, 2, 2, 0, 2, 0, 0, 0, 0, 1, 2, 1, 1, 0, 0, 0,
       2, 1, 1, 1, 2, 0, 0, 1, 1, 0, 0, 2, 1, 2, 0, 2, 1, 0, 2, 0, 0, 1,
       2, 1, 2, 2, 0, 0, 2, 0, 1, 2, 0, 2, 2, 2, 0, 1, 0, 2, 0, 0, 1, 0,
       1, 2, 0, 0, 2, 1, 2, 1, 1])

In [22]:
dat['pred'] = pred
dat

,0,test,pred
0,SR1129,2,0
1,NRS185,2,2
2,NRS243,1,1
3,BCH-SA-04,0,0
4,504,1,1
...,...,...,...
158,CFBREBSa131,2,2
159,CFBREBSa133,1,1
160,NRS256,2,2
161,GA48963,1,1


In [23]:
proba1 = model1_over.predict_proba(X_test_over)
dat_proba1 = pd.DataFrame(proba1)

In [24]:
dat_proba1

,0,1,2
0,8.974422e-01,1.631184e-02,0.086246
1,2.993144e-05,1.100095e-02,0.988969
2,6.563049e-04,9.869603e-01,0.012383
3,9.991750e-01,3.741669e-06,0.000821
4,9.667562e-03,9.664202e-01,0.023912
...,...,...,...
158,3.620039e-07,7.913741e-07,0.999999
159,1.459399e-03,9.621805e-01,0.036360
160,8.193681e-06,1.064882e-02,0.989343
161,2.966928e-06,9.835654e-01,0.016432


In [25]:
dat_proba1.to_csv("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/proba1.csv", index = False,
         header=None)

In [26]:
dat.to_csv("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/1p11p.csv", index = False,
         header=None)

In [41]:
hist1_over = model1_over.fit(X_train_over, y_train_over,
          batch_size=32, epochs=100,
          validation_data=(X_test_over, y_test_over))

Train on 380 samples, validate on 163 samples
Epoch 1/100
380/380 [==============================] - 0s 163us/step - loss: 0.1164 - accuracy: 0.9553 - val_loss: 0.4266 - val_accuracy: 0.8896
Epoch 2/100
380/380 [==============================] - 0s 144us/step - loss: 0.1273 - accuracy: 0.9474 - val_loss: 0.4278 - val_accuracy: 0.8896
Epoch 3/100
380/380 [==============================] - 0s 115us/step - loss: 0.1513 - accuracy: 0.9289 - val_loss: 0.4885 - val_accuracy: 0.8589
Epoch 4/100
380/380 [==============================] - 0s 107us/step - loss: 0.1330 - accuracy: 0.9447 - val_loss: 0.5197 - val_accuracy: 0.8528
Epoch 5/100
380/380 [==============================] - 0s 107us/step - loss: 0.1301 - accuracy: 0.9474 - val_loss: 0.5138 - val_accuracy: 0.8160
Epoch 6/100
380/380 [==============================] - 0s 108us/step - loss: 0.1491 - accuracy: 0.9342 - val_loss: 0.4683 - val_accuracy: 0.8466
Epoch 7/100
380/380 [==============================] - 0s 107us/step - loss: 0.1336 

/Users/Rebecca/anaconda3/lib/python3.7/site-packages/keras/callbacks/callbacks.py:95: RuntimeWarning: Method (on_train_batch_end) is slow compared to the batch update (0.112898). Check your callbacks.
  % (hook_name, delta_t_median), RuntimeWarning)


380/380 [==============================] - 0s 1ms/step - loss: 0.0919 - accuracy: 0.9711 - val_loss: 0.4612 - val_accuracy: 0.8773
Epoch 38/100
380/380 [==============================] - 0s 178us/step - loss: 0.1058 - accuracy: 0.9579 - val_loss: 0.4572 - val_accuracy: 0.8650
Epoch 39/100
380/380 [==============================] - 0s 171us/step - loss: 0.1003 - accuracy: 0.9632 - val_loss: 0.4499 - val_accuracy: 0.8834
Epoch 40/100
380/380 [==============================] - 0s 187us/step - loss: 0.0981 - accuracy: 0.9763 - val_loss: 0.5676 - val_accuracy: 0.8528
Epoch 41/100
380/380 [==============================] - 0s 113us/step - loss: 0.1318 - accuracy: 0.9553 - val_loss: 0.4464 - val_accuracy: 0.8834
Epoch 42/100
380/380 [==============================] - 0s 107us/step - loss: 0.1309 - accuracy: 0.9553 - val_loss: 0.4567 - val_accuracy: 0.8773
Epoch 43/100
380/380 [==============================] - 0s 297us/step - loss: 0.1054 - accuracy: 0.9605 - val_loss: 0.4406 - val_accuracy: 

In [42]:
print('over-sampling train accuracy: %.2f%%' % (np.mean(hist1_over.history['accuracy'])*100))

over-sampling train accuracy: 95.91%


In [18]:
df_proba = pd.read_excel("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/NN_over_2.xlsx",
                        sheet_name=0,
                        index_col=None)

In [19]:
df_proba

,phage,strain,phenotype,prediction,0,1,2
0,p002ykpresabs_qual,CFBRSa26,0,0,0.758914,0.241086,4.638713e-07
1,p002ykpresabs_qual,NRS109,2,2,0.005361,0.016236,9.784034e-01
2,p002ykpresabs_qual,NRS112,0,0,0.726623,0.273376,1.520979e-06
3,p002ykpresabs_qual,NRS216,1,1,0.138322,0.861665,1.334123e-05
4,p002ykpresabs_qual,NRS021,0,0,0.882176,0.117824,1.414530e-10
...,...,...,...,...,...,...,...
4279,pyopresabsSTCC_qual,NRS148,2,2,0.000007,0.000099,9.998934e-01
4280,pyopresabsSTCC_qual,NRS255,2,2,0.000257,0.002048,9.976944e-01
4281,pyopresabsSTCC_qual,NRS205,2,2,0.000011,0.000045,9.999435e-01
4282,pyopresabsSTCC_qual,NRS255,2,2,0.000257,0.002048,9.976944e-01


In [20]:
y_prob = df_proba[df_proba['phage']=='p11presabs_qual'].iloc[:,-3:]
y_prob = y_prob.to_numpy()
y_prob

array([[8.97442200e-01, 1.63118400e-02, 8.62458350e-02],
       [2.99314380e-05, 1.10009540e-02, 9.88969100e-01],
       [6.56304900e-04, 9.86960300e-01, 1.23833580e-02],
       [9.99175000e-01, 3.74166920e-06, 8.21327700e-04],
       [9.66756200e-03, 9.66420230e-01, 2.39122250e-02],
       [8.64572600e-01, 8.71616300e-02, 4.82658300e-02],
       [9.93526000e-01, 3.33350450e-03, 3.14053820e-03],
       [8.65304700e-01, 5.64754120e-05, 1.34638950e-01],
       [1.83631820e-02, 7.75294000e-01, 2.06342790e-01],
       [5.09233630e-03, 9.89207500e-01, 5.70020170e-03],
       [5.66357340e-05, 9.59045700e-03, 9.90352900e-01],
       [9.90899440e-01, 8.96445600e-03, 1.36130740e-04],
       [8.72450050e-01, 9.89174900e-02, 2.86324640e-02],
       [7.33481100e-04, 1.03320780e-02, 9.88934460e-01],
       [9.90899440e-01, 8.96445600e-03, 1.36130740e-04],
       [9.95040240e-01, 4.47050170e-04, 4.51269650e-03],
       [1.48833220e-04, 2.67485730e-02, 9.73102600e-01],
       [1.06803660e-02, 8.55337

In [21]:
## Retrieved from https://github.com/scikit-learn/scikit-learn/issues/3298
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import LabelBinarizer

def rocauc_ovo(truth, pred, average="macro", multi_class="ovo"):

    lb = LabelBinarizer()
    lb.fit(truth)

    truth = lb.transform(truth)   
    
    return roc_auc_score(truth, pred, average=average, multi_class=multi_class)

In [22]:
ovo1 = rocauc_ovo(y_test_over, y_prob, average="macro", multi_class="ovo")
ovo1

0.9676716193535766

In [23]:
def rocauc_ovr(truth, pred, average="macro", multi_class="ovr"):

    lb = LabelBinarizer()
    lb.fit(truth)

    truth = lb.transform(truth)   

    return roc_auc_score(truth, pred, average=average, multi_class=multi_class)

In [24]:
ovr1 = rocauc_ovr(y_test_over, y_prob, average="macro", multi_class="ovr")
ovr1

0.9676716193535766

In [25]:
# split into train, test data (over)
from sklearn.model_selection import train_test_split
X_train_over, X_test_over, y_train_over, y_test_over = train_test_split(X_over, y_over,
                                                    test_size = 0.3,
                                                    random_state=234,
                                                    stratify=y_over)

In [26]:
dat2 = pd.DataFrame(X_test_over[:,0])
dat2['test'] = y_test_over

In [27]:
dat2

,0,test
0,NRS027,0
1,CFBRSa07,0
2,CFBRSa27,1
3,504,1
4,CFBREBSa129,0
...,...,...
158,SR3569,2
159,NRS243,1
160,GA48963,1
161,504,1


In [28]:
X_train_over = X_train_over[:,1:]
X_test_over = X_test_over[:,1:]

In [29]:
model1_over2 = Sequential([
    Dense(32, activation='relu', input_shape=(X_train_over.shape[1],)),
    Dense(3, activation='softmax'),
])

In [32]:
model1_over2.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [33]:
model1_over2.fit(X_train_over, y_train_over,
          batch_size=32, epochs=100,
          validation_data=(X_test_over, y_test_over))

Train on 380 samples, validate on 163 samples
Epoch 1/100
380/380 [==============================] - 0s 372us/step - loss: 1.1094 - accuracy: 0.4237 - val_loss: 1.0615 - val_accuracy: 0.5153
Epoch 2/100
380/380 [==============================] - 0s 172us/step - loss: 0.9539 - accuracy: 0.5368 - val_loss: 0.9731 - val_accuracy: 0.4601
Epoch 3/100
380/380 [==============================] - 0s 169us/step - loss: 0.8438 - accuracy: 0.5947 - val_loss: 0.9613 - val_accuracy: 0.5828
Epoch 4/100
380/380 [==============================] - 0s 162us/step - loss: 0.8118 - accuracy: 0.6263 - val_loss: 0.8207 - val_accuracy: 0.5828
Epoch 5/100
380/380 [==============================] - 0s 279us/step - loss: 0.7041 - accuracy: 0.6842 - val_loss: 0.7843 - val_accuracy: 0.5767
Epoch 6/100
380/380 [==============================] - 0s 157us/step - loss: 0.6801 - accuracy: 0.6868 - val_loss: 0.8201 - val_accuracy: 0.6258
Epoch 7/100
380/380 [==============================] - 0s 168us/step - loss: 0.6499 

Epoch 57/100
380/380 [==============================] - 0s 124us/step - loss: 0.1772 - accuracy: 0.9474 - val_loss: 0.3161 - val_accuracy: 0.9141
Epoch 58/100
380/380 [==============================] - 0s 120us/step - loss: 0.1791 - accuracy: 0.9316 - val_loss: 0.3075 - val_accuracy: 0.8896
Epoch 59/100
380/380 [==============================] - 0s 105us/step - loss: 0.1916 - accuracy: 0.9368 - val_loss: 0.2902 - val_accuracy: 0.8957
Epoch 60/100
380/380 [==============================] - 0s 110us/step - loss: 0.1857 - accuracy: 0.9316 - val_loss: 0.3364 - val_accuracy: 0.9018
Epoch 61/100
380/380 [==============================] - 0s 104us/step - loss: 0.1769 - accuracy: 0.9316 - val_loss: 0.3201 - val_accuracy: 0.9141
Epoch 62/100
380/380 [==============================] - 0s 108us/step - loss: 0.1654 - accuracy: 0.9526 - val_loss: 0.3470 - val_accuracy: 0.8712
Epoch 63/100
380/380 [==============================] - 0s 103us/step - loss: 0.1876 - accuracy: 0.9395 - val_loss: 0.3068 -

In [50]:
acc_test_over2 = model1_over2.evaluate(X_test_over, y_test_over)[1]
print('over-sampling test accuracy: %.2f%%' % (acc_test_over2*100))

163/163 [==============================] - 0s 106us/step
over-sampling test accuracy: 90.18%


In [34]:
pred2 = model1_over2.predict_classes(X_test_over)
pred2

array([0, 0, 0, 2, 0, 0, 0, 1, 0, 1, 2, 0, 1, 1, 1, 2, 1, 1, 0, 2, 1, 0,
       1, 0, 2, 0, 1, 0, 1, 0, 0, 0, 0, 2, 1, 0, 2, 1, 0, 0, 0, 0, 2, 1,
       1, 0, 0, 2, 1, 0, 0, 0, 0, 1, 0, 0, 2, 0, 0, 0, 2, 1, 2, 0, 2, 1,
       0, 2, 1, 2, 2, 0, 1, 2, 2, 2, 2, 0, 0, 2, 1, 2, 1, 0, 2, 1, 2, 0,
       2, 2, 2, 0, 1, 0, 0, 1, 0, 2, 2, 0, 0, 0, 1, 0, 1, 2, 1, 2, 1, 0,
       1, 1, 0, 0, 1, 2, 2, 1, 1, 0, 0, 1, 2, 2, 1, 0, 1, 0, 0, 2, 0, 0,
       2, 0, 2, 2, 1, 0, 0, 2, 1, 2, 0, 2, 0, 0, 0, 2, 2, 2, 1, 0, 0, 2,
       1, 1, 0, 2, 2, 1, 1, 2, 0])

In [35]:
dat2['pred'] = pred2
dat2

,0,test,pred
0,NRS027,0,0
1,CFBRSa07,0,0
2,CFBRSa27,1,0
3,504,1,2
4,CFBREBSa129,0,0
...,...,...,...
158,SR3569,2,2
159,NRS243,1,1
160,GA48963,1,1
161,504,1,2


In [36]:
proba2 = model1_over2.predict_proba(X_test_over)
dat_proba2 = pd.DataFrame(proba2)

In [37]:
dat_proba2

,0,1,2
0,0.999576,0.000147,0.000277
1,0.998123,0.000037,0.001840
2,0.786800,0.097460,0.115741
3,0.311864,0.307633,0.380503
4,0.999031,0.000525,0.000444
...,...,...,...
158,0.043488,0.115955,0.840557
159,0.000020,0.988498,0.011483
160,0.000330,0.787300,0.212369
161,0.311864,0.307633,0.380503


In [38]:
dat_proba2.to_csv("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/proba2.csv", index = False,
         header=None)

In [39]:
dat2.to_csv("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/2p11p.csv", index = False,
         header=None)

In [54]:
hist1_over2 = model1_over2.fit(X_train_over, y_train_over,
          batch_size=32, epochs=100,
          validation_data=(X_test_over, y_test_over))

Train on 380 samples, validate on 163 samples
Epoch 1/100
380/380 [==============================] - 0s 166us/step - loss: 0.1152 - accuracy: 0.9658 - val_loss: 0.3429 - val_accuracy: 0.8957
Epoch 2/100
380/380 [==============================] - 0s 120us/step - loss: 0.1111 - accuracy: 0.9684 - val_loss: 0.3723 - val_accuracy: 0.8896
Epoch 3/100
380/380 [==============================] - 0s 129us/step - loss: 0.1146 - accuracy: 0.9579 - val_loss: 0.2945 - val_accuracy: 0.9141
Epoch 4/100
380/380 [==============================] - 0s 110us/step - loss: 0.1142 - accuracy: 0.9605 - val_loss: 0.3076 - val_accuracy: 0.8957
Epoch 5/100
380/380 [==============================] - 0s 107us/step - loss: 0.1091 - accuracy: 0.9632 - val_loss: 0.2894 - val_accuracy: 0.9141
Epoch 6/100
380/380 [==============================] - 0s 119us/step - loss: 0.1100 - accuracy: 0.9632 - val_loss: 0.3068 - val_accuracy: 0.9018
Epoch 7/100
380/380 [==============================] - 0s 107us/step - loss: 0.1036 

Epoch 57/100
380/380 [==============================] - 0s 108us/step - loss: 0.1212 - accuracy: 0.9579 - val_loss: 0.3645 - val_accuracy: 0.8896
Epoch 58/100
380/380 [==============================] - 0s 116us/step - loss: 0.0853 - accuracy: 0.9763 - val_loss: 0.4644 - val_accuracy: 0.8834
Epoch 59/100
380/380 [==============================] - 0s 108us/step - loss: 0.1146 - accuracy: 0.9605 - val_loss: 0.3138 - val_accuracy: 0.9018
Epoch 60/100
380/380 [==============================] - 0s 107us/step - loss: 0.1224 - accuracy: 0.9474 - val_loss: 0.3808 - val_accuracy: 0.8957
Epoch 61/100
380/380 [==============================] - 0s 106us/step - loss: 0.1208 - accuracy: 0.9605 - val_loss: 0.3385 - val_accuracy: 0.9080
Epoch 62/100
380/380 [==============================] - 0s 104us/step - loss: 0.1038 - accuracy: 0.9553 - val_loss: 0.3072 - val_accuracy: 0.9202
Epoch 63/100
380/380 [==============================] - 0s 112us/step - loss: 0.1041 - accuracy: 0.9632 - val_loss: 0.3233 -

In [55]:
print('over-sampling train accuracy: %.2f%%' % (np.mean(hist1_over2.history['accuracy'])*100))

over-sampling train accuracy: 96.27%


In [30]:
df_proba2 = pd.read_excel("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/NN_over_2.xlsx",
                        sheet_name=1,
                        index_col=None)

In [31]:
df_proba2

,phage,strain,phenotype,prediction,0,1,2
0,p002ykpresabs_qual,NRS148,2,2,0.000056,1.748042e-03,9.981960e-01
1,p002ykpresabs_qual,BCH-SA-03,1,0,0.712007,2.879924e-01,9.646217e-07
2,p002ykpresabs_qual,NRS218,1,1,0.006222,9.937732e-01,4.482882e-06
3,p002ykpresabs_qual,NRS036,0,0,0.882617,1.173831e-01,2.310933e-10
4,p002ykpresabs_qual,NRS386,1,0,0.571179,4.288184e-01,2.444667e-06
...,...,...,...,...,...,...,...
4279,pyopresabsSTCC_qual,NRS112,1,1,0.001860,9.979747e-01,1.653396e-04
4280,pyopresabsSTCC_qual,SR1065,0,0,0.982940,1.705227e-02,7.349168e-06
4281,pyopresabsSTCC_qual,NRS203,0,0,0.997093,1.962516e-03,9.441347e-04
4282,pyopresabsSTCC_qual,CFBREBSa129,0,0,1.000000,3.031141e-13,3.208205e-09


In [32]:
y_prob2 = df_proba2[df_proba2['phage']=='p11presabs_qual'].iloc[:,-3:]
y_prob2 = y_prob2.to_numpy()
y_prob2

array([[9.99576300e-01, 1.47134720e-04, 2.76561740e-04],
       [9.98123200e-01, 3.70915080e-05, 1.83963000e-03],
       [7.86799600e-01, 9.74595900e-02, 1.15740790e-01],
       [3.11864170e-01, 3.07633220e-01, 3.80502670e-01],
       [9.99031200e-01, 5.24683600e-04, 4.44119880e-04],
       [9.86831070e-01, 3.20975900e-05, 1.31367890e-02],
       [9.99031200e-01, 5.24683600e-04, 4.44119880e-04],
       [2.43028400e-02, 6.10687900e-01, 3.65009250e-01],
       [7.86799600e-01, 9.74595900e-02, 1.15740790e-01],
       [8.78673050e-03, 9.86969950e-01, 4.24322900e-03],
       [1.06192560e-08, 3.17531560e-05, 9.99968300e-01],
       [9.98318850e-01, 1.28406790e-03, 3.96977470e-04],
       [1.45408140e-02, 7.86452200e-01, 1.99007030e-01],
       [9.76358900e-02, 6.30768500e-01, 2.71595600e-01],
       [1.50144810e-05, 9.86956950e-01, 1.30279360e-02],
       [6.05480560e-14, 1.36032190e-09, 1.00000000e+00],
       [4.85778400e-06, 8.92515400e-01, 1.07479620e-01],
       [2.10603400e-02, 9.01900

In [33]:
ovo2 = rocauc_ovo(y_test_over, y_prob2, average="macro", multi_class="ovo")
ovo2

0.9735566675933649

In [34]:
ovr2 = rocauc_ovr(y_test_over, y_prob2, average="macro", multi_class="ovr")
ovr2

0.9735566675933649

In [35]:
# split into train, test data (over)
from sklearn.model_selection import train_test_split
X_train_over, X_test_over, y_train_over, y_test_over = train_test_split(X_over, y_over,
                                                    test_size = 0.3,
                                                    random_state=345,
                                                    stratify=y_over)

In [36]:
dat3 = pd.DataFrame(X_test_over[:,0])
dat3['test'] = y_test_over

In [37]:
dat3

,0,test
0,NRS149,0
1,EUH13,0
2,NRS106,2
3,NRS214,1
4,CFBREBSa129,0
...,...,...
158,NRS027,0
159,CFBRSa70,2
160,CFBREBSa130,0
161,NRS214,1


In [38]:
X_train_over = X_train_over[:,1:]
X_test_over = X_test_over[:,1:]

In [44]:
model1_over3 = Sequential([
    Dense(32, activation='relu', input_shape=(X_train_over.shape[1],)),
    Dense(3, activation='softmax'),
])

In [45]:
model1_over3.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [46]:
model1_over3.fit(X_train_over, y_train_over,
          batch_size=32, epochs=100,
          validation_data=(X_test_over, y_test_over))

Train on 380 samples, validate on 163 samples
Epoch 1/100
380/380 [==============================] - 0s 376us/step - loss: 1.0742 - accuracy: 0.4368 - val_loss: 0.9354 - val_accuracy: 0.5583
Epoch 2/100
380/380 [==============================] - 0s 160us/step - loss: 0.8840 - accuracy: 0.5921 - val_loss: 0.8311 - val_accuracy: 0.6871
Epoch 3/100
380/380 [==============================] - 0s 166us/step - loss: 0.8074 - accuracy: 0.6421 - val_loss: 0.7803 - val_accuracy: 0.6074
Epoch 4/100
380/380 [==============================] - 0s 186us/step - loss: 0.7470 - accuracy: 0.6526 - val_loss: 0.7290 - val_accuracy: 0.6564
Epoch 5/100
380/380 [==============================] - 0s 166us/step - loss: 0.6966 - accuracy: 0.6605 - val_loss: 0.7041 - val_accuracy: 0.6994
Epoch 6/100
380/380 [==============================] - 0s 168us/step - loss: 0.6481 - accuracy: 0.7263 - val_loss: 0.6580 - val_accuracy: 0.7301
Epoch 7/100
380/380 [==============================] - 0s 157us/step - loss: 0.5962 

Epoch 57/100
380/380 [==============================] - 0s 149us/step - loss: 0.1402 - accuracy: 0.9711 - val_loss: 0.4537 - val_accuracy: 0.8650
Epoch 58/100
380/380 [==============================] - 0s 204us/step - loss: 0.1281 - accuracy: 0.9737 - val_loss: 0.3668 - val_accuracy: 0.8834
Epoch 59/100
380/380 [==============================] - 0s 242us/step - loss: 0.1369 - accuracy: 0.9632 - val_loss: 0.3803 - val_accuracy: 0.8712
Epoch 60/100
380/380 [==============================] - 0s 270us/step - loss: 0.1294 - accuracy: 0.9711 - val_loss: 0.3667 - val_accuracy: 0.8712
Epoch 61/100
380/380 [==============================] - 0s 144us/step - loss: 0.1286 - accuracy: 0.9658 - val_loss: 0.3577 - val_accuracy: 0.9018
Epoch 62/100
380/380 [==============================] - 0s 124us/step - loss: 0.1220 - accuracy: 0.9763 - val_loss: 0.3518 - val_accuracy: 0.9018
Epoch 63/100
380/380 [==============================] - 0s 122us/step - loss: 0.1451 - accuracy: 0.9632 - val_loss: 0.3487 -

In [63]:
acc_test_over3 = model1_over3.evaluate(X_test_over, y_test_over)[1]
print('over-sampling test accuracy: %.2f%%' % (acc_test_over3*100))

163/163 [==============================] - 0s 96us/step
over-sampling test accuracy: 90.18%


In [47]:
pred3 = model1_over3.predict_classes(X_test_over)
pred3

array([0, 0, 0, 1, 0, 2, 0, 0, 2, 0, 1, 2, 1, 2, 0, 1, 1, 0, 0, 2, 1, 2,
       0, 2, 2, 1, 2, 2, 0, 0, 1, 1, 1, 0, 2, 1, 1, 2, 1, 2, 0, 1, 1, 1,
       2, 0, 2, 1, 0, 2, 2, 0, 0, 0, 1, 1, 2, 1, 2, 1, 2, 1, 0, 1, 1, 0,
       0, 0, 0, 2, 0, 1, 2, 0, 0, 1, 0, 2, 0, 0, 0, 1, 2, 2, 2, 0, 0, 1,
       2, 0, 0, 0, 0, 0, 1, 2, 1, 0, 0, 1, 2, 0, 1, 1, 1, 0, 2, 1, 1, 1,
       0, 1, 0, 0, 2, 2, 1, 1, 1, 2, 2, 2, 0, 2, 1, 1, 2, 1, 1, 1, 1, 0,
       1, 2, 0, 2, 0, 0, 0, 2, 1, 0, 1, 0, 1, 1, 0, 0, 0, 2, 0, 2, 1, 0,
       1, 0, 1, 1, 0, 1, 0, 1, 1])

In [48]:
dat3['pred'] = pred3
dat3

,0,test,pred
0,NRS149,0,0
1,EUH13,0,0
2,NRS106,2,0
3,NRS214,1,1
4,CFBREBSa129,0,0
...,...,...,...
158,NRS027,0,0
159,CFBRSa70,2,1
160,CFBREBSa130,0,0
161,NRS214,1,1


In [49]:
proba3 = model1_over3.predict_proba(X_test_over)
dat_proba3 = pd.DataFrame(proba3)

In [50]:
dat_proba3

,0,1,2
0,9.930519e-01,0.003753,0.003196
1,9.874885e-01,0.011873,0.000639
2,9.017478e-01,0.073918,0.024334
3,2.036675e-02,0.939113,0.040520
4,9.980913e-01,0.001881,0.000028
...,...,...,...
158,9.989587e-01,0.001020,0.000021
159,2.886204e-01,0.697115,0.014265
160,9.944845e-01,0.000317,0.005198
161,2.036675e-02,0.939113,0.040520


In [51]:
dat_proba3.to_csv("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/proba3.csv", index = False,
         header=None)

In [52]:
dat3.to_csv("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/3p11p.csv", index = False,
         header=None)

In [67]:
hist1_over3 = model1_over3.fit(X_train_over, y_train_over,
          batch_size=32, epochs=100,
          validation_data=(X_test_over, y_test_over))

Train on 380 samples, validate on 163 samples
Epoch 1/100
380/380 [==============================] - 0s 157us/step - loss: 0.1104 - accuracy: 0.9684 - val_loss: 0.3017 - val_accuracy: 0.9080
Epoch 2/100
380/380 [==============================] - 0s 123us/step - loss: 0.1138 - accuracy: 0.9632 - val_loss: 0.3227 - val_accuracy: 0.8834
Epoch 3/100
380/380 [==============================] - 0s 113us/step - loss: 0.1030 - accuracy: 0.9711 - val_loss: 0.3063 - val_accuracy: 0.9080
Epoch 4/100
380/380 [==============================] - 0s 122us/step - loss: 0.1039 - accuracy: 0.9737 - val_loss: 0.3373 - val_accuracy: 0.8896
Epoch 5/100
380/380 [==============================] - 0s 115us/step - loss: 0.0858 - accuracy: 0.9816 - val_loss: 0.3640 - val_accuracy: 0.8896
Epoch 6/100
380/380 [==============================] - 0s 108us/step - loss: 0.0963 - accuracy: 0.9763 - val_loss: 0.5135 - val_accuracy: 0.8466
Epoch 7/100
380/380 [==============================] - 0s 116us/step - loss: 0.1192 

Epoch 57/100
380/380 [==============================] - 0s 115us/step - loss: 0.0807 - accuracy: 0.9684 - val_loss: 0.4047 - val_accuracy: 0.8834
Epoch 58/100
380/380 [==============================] - 0s 116us/step - loss: 0.0623 - accuracy: 0.9816 - val_loss: 0.4913 - val_accuracy: 0.8589
Epoch 59/100
380/380 [==============================] - 0s 116us/step - loss: 0.0563 - accuracy: 0.9816 - val_loss: 0.4745 - val_accuracy: 0.8712
Epoch 60/100
380/380 [==============================] - 0s 122us/step - loss: 0.0596 - accuracy: 0.9816 - val_loss: 0.5148 - val_accuracy: 0.8650
Epoch 61/100
380/380 [==============================] - 0s 107us/step - loss: 0.0583 - accuracy: 0.9816 - val_loss: 0.5186 - val_accuracy: 0.8650
Epoch 62/100
380/380 [==============================] - 0s 111us/step - loss: 0.1091 - accuracy: 0.9553 - val_loss: 0.7576 - val_accuracy: 0.8528
Epoch 63/100
380/380 [==============================] - 0s 117us/step - loss: 0.1060 - accuracy: 0.9579 - val_loss: 0.4444 -

In [68]:
print('over-sampling train accuracy: %.2f%%' % (np.mean(hist1_over3.history['accuracy'])*100))

over-sampling train accuracy: 97.51%


In [39]:
df_proba3 = pd.read_excel("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/NN_over_2.xlsx",
                        sheet_name=2,
                        index_col=None)

In [40]:
df_proba3

,phage,strain,phenotype,prediction,0,1,2
0,p002ykpresabs_qual,NRS109,2,2,0.004477,0.013518,9.820048e-01
1,p002ykpresabs_qual,NRS109,2,2,0.004477,0.013518,9.820048e-01
2,p002ykpresabs_qual,NRS222,0,0,0.851725,0.148269,5.980786e-06
3,p002ykpresabs_qual,NRS109,2,2,0.004477,0.013518,9.820048e-01
4,p002ykpresabs_qual,GA50245,0,0,0.812055,0.187945,1.161034e-07
...,...,...,...,...,...,...,...
4279,pyopresabsSTCC_qual,NRS255,2,2,0.000633,0.000928,9.984396e-01
4280,pyopresabsSTCC_qual,NRS255,2,2,0.000633,0.000928,9.984396e-01
4281,pyopresabsSTCC_qual,NRS266,1,1,0.025932,0.974061,7.323514e-06
4282,pyopresabsSTCC_qual,NRS001,1,1,0.000597,0.999403,3.675362e-10


In [41]:
y_prob3 = df_proba3[df_proba3['phage']=='p11presabs_qual'].iloc[:,-3:]
y_prob3 = y_prob3.to_numpy()
y_prob3

array([[9.9305190e-01, 3.7526863e-03, 3.1955044e-03],
       [9.8748850e-01, 1.1872638e-02, 6.3884980e-04],
       [9.0174776e-01, 7.3917750e-02, 2.4334421e-02],
       [2.0366747e-02, 9.3911320e-01, 4.0519953e-02],
       [9.9809130e-01, 1.8808233e-03, 2.7855504e-05],
       [1.0060513e-03, 2.3835833e-01, 7.6063570e-01],
       [9.9895870e-01, 1.0201789e-03, 2.1116744e-05],
       [7.8345780e-01, 1.5822506e-01, 5.8317162e-02],
       [1.6870590e-03, 2.4654175e-04, 9.9806637e-01],
       [9.5704810e-01, 4.2899497e-02, 5.2420000e-05],
       [4.5415204e-02, 9.0583116e-01, 4.8753670e-02],
       [9.2836140e-07, 2.9438186e-05, 9.9996960e-01],
       [1.4491033e-06, 9.6917510e-01, 3.0823348e-02],
       [1.3627746e-11, 4.5725750e-08, 1.0000000e+00],
       [9.9317825e-01, 6.4289668e-03, 3.9270805e-04],
       [1.1150439e-04, 9.7580760e-01, 2.4080833e-02],
       [3.6498480e-02, 7.9646000e-01, 1.6704161e-01],
       [7.8345780e-01, 1.5822506e-01, 5.8317162e-02],
       [9.9268370e-01, 3.843

In [42]:
ovo3 = rocauc_ovo(y_test_over, y_prob3, average="macro", multi_class="ovo")
ovo3

0.9647581832597124

In [43]:
ovr3 = rocauc_ovr(y_test_over, y_prob3, average="macro", multi_class="ovr")
ovr3

0.9647581832597124

In [44]:
# split into train, test data (over)
from sklearn.model_selection import train_test_split
X_train_over, X_test_over, y_train_over, y_test_over = train_test_split(X_over, y_over,
                                                    test_size = 0.3,
                                                    random_state=456,
                                                    stratify=y_over)

In [45]:
dat4 = pd.DataFrame(X_test_over[:,0])
dat4['test'] = y_test_over

In [46]:
dat4

,0,test
0,SR2852,2
1,CFBREBSa138,0
2,BCH-SA-12,0
3,EUH13,0
4,EUH13,0
...,...,...
158,NRS036,1
159,CA105,1
160,CFBRSa51,1
161,NRS102,1


In [47]:
X_train_over = X_train_over[:,1:]
X_test_over = X_test_over[:,1:]

In [48]:
model1_over4 = Sequential([
    Dense(32, activation='relu', input_shape=(X_train_over.shape[1],)),
    Dense(32, activation='relu'),
    Dense(3, activation='softmax'),
])

In [58]:
model1_over4.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [59]:
model1_over4.fit(X_train_over, y_train_over,
          batch_size=64, epochs=100,
          validation_data=(X_test_over, y_test_over))

Train on 380 samples, validate on 163 samples
Epoch 1/100
380/380 [==============================] - 0s 597us/step - loss: 1.1719 - accuracy: 0.3763 - val_loss: 1.0926 - val_accuracy: 0.3804
Epoch 2/100
380/380 [==============================] - 0s 169us/step - loss: 1.0448 - accuracy: 0.4579 - val_loss: 1.0213 - val_accuracy: 0.5890
Epoch 3/100
380/380 [==============================] - 0s 173us/step - loss: 0.9944 - accuracy: 0.5474 - val_loss: 0.9780 - val_accuracy: 0.5644
Epoch 4/100
380/380 [==============================] - 0s 165us/step - loss: 0.9439 - accuracy: 0.5737 - val_loss: 0.9402 - val_accuracy: 0.5521
Epoch 5/100
380/380 [==============================] - 0s 167us/step - loss: 0.8850 - accuracy: 0.5895 - val_loss: 0.9192 - val_accuracy: 0.5706
Epoch 6/100
380/380 [==============================] - 0s 142us/step - loss: 0.8323 - accuracy: 0.6342 - val_loss: 0.8688 - val_accuracy: 0.5951
Epoch 7/100
380/380 [==============================] - 0s 150us/step - loss: 0.7825 

Epoch 57/100
380/380 [==============================] - 0s 168us/step - loss: 0.1648 - accuracy: 0.9474 - val_loss: 0.3580 - val_accuracy: 0.9080
Epoch 58/100
380/380 [==============================] - 0s 205us/step - loss: 0.1630 - accuracy: 0.9421 - val_loss: 0.3366 - val_accuracy: 0.9018
Epoch 59/100
380/380 [==============================] - 0s 230us/step - loss: 0.1525 - accuracy: 0.9526 - val_loss: 0.3427 - val_accuracy: 0.9018
Epoch 60/100
380/380 [==============================] - 0s 97us/step - loss: 0.1524 - accuracy: 0.9605 - val_loss: 0.3843 - val_accuracy: 0.8834
Epoch 61/100
380/380 [==============================] - 0s 99us/step - loss: 0.1537 - accuracy: 0.9500 - val_loss: 0.3457 - val_accuracy: 0.8834
Epoch 62/100
380/380 [==============================] - 0s 83us/step - loss: 0.1485 - accuracy: 0.9553 - val_loss: 0.4171 - val_accuracy: 0.8589
Epoch 63/100
380/380 [==============================] - 0s 89us/step - loss: 0.1782 - accuracy: 0.9316 - val_loss: 0.3320 - val

In [92]:
acc_test_over4 = model1_over4.evaluate(X_test_over, y_test_over)[1]
print('over-sampling test accuracy: %.2f%%' % (acc_test_over4*100))

163/163 [==============================] - 0s 106us/step
over-sampling test accuracy: 91.41%


In [60]:
pred4 = model1_over4.predict_classes(X_test_over)
pred4

array([2, 0, 0, 0, 0, 1, 0, 2, 0, 0, 1, 0, 2, 0, 2, 0, 0, 0, 2, 0, 2, 2,
       0, 1, 1, 1, 2, 0, 1, 0, 0, 2, 1, 1, 0, 0, 0, 2, 0, 0, 0, 1, 1, 1,
       0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 2, 1, 2, 1, 1, 0, 1, 1, 0, 2, 1, 1,
       1, 0, 2, 0, 0, 1, 0, 0, 1, 2, 1, 1, 1, 2, 0, 0, 2, 1, 2, 1, 0, 1,
       0, 0, 0, 1, 0, 2, 1, 1, 0, 1, 1, 1, 0, 0, 2, 1, 2, 2, 0, 2, 1, 0,
       2, 0, 1, 1, 0, 2, 0, 1, 2, 1, 1, 1, 2, 2, 1, 1, 2, 1, 2, 0, 1, 0,
       1, 1, 2, 1, 0, 2, 1, 1, 0, 0, 2, 0, 1, 2, 1, 1, 1, 0, 0, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 2])

In [61]:
dat4['pred'] = pred4
dat4

,0,test,pred
0,SR2852,2,2
1,CFBREBSa138,0,0
2,BCH-SA-12,0,0
3,EUH13,0,0
4,EUH13,0,0
...,...,...,...
158,NRS036,1,1
159,CA105,1,1
160,CFBRSa51,1,1
161,NRS102,1,1


In [62]:
proba4 = model1_over4.predict_proba(X_test_over)
dat_proba4 = pd.DataFrame(proba4)

In [63]:
dat_proba4

,0,1,2
0,1.802514e-06,1.108062e-03,0.998890
1,9.996647e-01,1.931857e-04,0.000142
2,9.992285e-01,6.816303e-04,0.000090
3,9.884464e-01,1.041394e-02,0.001140
4,9.884464e-01,1.041394e-02,0.001140
...,...,...,...
158,1.051734e-02,9.504162e-01,0.039066
159,2.425348e-05,9.996481e-01,0.000328
160,1.810323e-04,9.889921e-01,0.010827
161,6.294157e-05,9.933652e-01,0.006572


In [64]:
dat_proba4.to_csv("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/proba4.csv", index = False,
         header=None)

In [65]:
dat4.to_csv("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/4p11p.csv", index = False,
         header=None)

In [96]:
hist1_over4 = model1_over4.fit(X_train_over, y_train_over,
          batch_size=64, epochs=100,
          validation_data=(X_test_over, y_test_over))

Train on 380 samples, validate on 163 samples
Epoch 1/100
380/380 [==============================] - 0s 148us/step - loss: 0.0896 - accuracy: 0.9605 - val_loss: 0.4265 - val_accuracy: 0.8773
Epoch 2/100
380/380 [==============================] - 0s 120us/step - loss: 0.0904 - accuracy: 0.9658 - val_loss: 0.3583 - val_accuracy: 0.8957
Epoch 3/100
380/380 [==============================] - 0s 105us/step - loss: 0.0880 - accuracy: 0.9737 - val_loss: 0.3198 - val_accuracy: 0.9080
Epoch 4/100
380/380 [==============================] - 0s 103us/step - loss: 0.0841 - accuracy: 0.9737 - val_loss: 0.3173 - val_accuracy: 0.9080
Epoch 5/100
380/380 [==============================] - 0s 113us/step - loss: 0.1026 - accuracy: 0.9605 - val_loss: 0.3054 - val_accuracy: 0.9141
Epoch 6/100
380/380 [==============================] - 0s 94us/step - loss: 0.1024 - accuracy: 0.9658 - val_loss: 0.2995 - val_accuracy: 0.9141
Epoch 7/100
380/380 [==============================] - 0s 89us/step - loss: 0.0765 - 

Epoch 57/100
380/380 [==============================] - 0s 102us/step - loss: 0.0614 - accuracy: 0.9842 - val_loss: 0.3637 - val_accuracy: 0.9080
Epoch 58/100
380/380 [==============================] - 0s 103us/step - loss: 0.0614 - accuracy: 0.9789 - val_loss: 0.5003 - val_accuracy: 0.8712
Epoch 59/100
380/380 [==============================] - 0s 89us/step - loss: 0.0997 - accuracy: 0.9553 - val_loss: 0.3494 - val_accuracy: 0.9080
Epoch 60/100
380/380 [==============================] - 0s 96us/step - loss: 0.0796 - accuracy: 0.9632 - val_loss: 0.3576 - val_accuracy: 0.9080
Epoch 61/100
380/380 [==============================] - 0s 93us/step - loss: 0.0750 - accuracy: 0.9632 - val_loss: 0.3585 - val_accuracy: 0.9080
Epoch 62/100
380/380 [==============================] - 0s 98us/step - loss: 0.0929 - accuracy: 0.9605 - val_loss: 0.3785 - val_accuracy: 0.9018
Epoch 63/100
380/380 [==============================] - 0s 101us/step - loss: 0.1233 - accuracy: 0.9553 - val_loss: 0.7003 - val

In [97]:
print('over-sampling train accuracy: %.2f%%' % (np.mean(hist1_over4.history['accuracy'])*100))

over-sampling train accuracy: 96.88%


In [49]:
df_proba4 = pd.read_excel("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/NN_over_2.xlsx",
                        sheet_name=3,
                        index_col=None)

In [50]:
df_proba4

,phage,strain,phenotype,prediction,0,1,2
0,p002ykpresabs_qual,NRS110,1,1,0.000003,0.999997,5.870196e-13
1,p002ykpresabs_qual,NRS216,1,1,0.039254,0.960745,9.078969e-07
2,p002ykpresabs_qual,NRS386,1,1,0.326752,0.673248,1.061032e-07
3,p002ykpresabs_qual,CFBRSa25,0,0,0.611084,0.388916,7.664974e-07
4,p002ykpresabs_qual,BCH-SA-03,1,0,0.611084,0.388916,7.664974e-07
...,...,...,...,...,...,...,...
4279,pyopresabsSTCC_qual,NRS236,1,1,0.000052,0.999768,1.803156e-04
4280,pyopresabsSTCC_qual,NRS029,0,1,0.322350,0.677496,1.533154e-04
4281,pyopresabsSTCC_qual,NRS148,2,2,0.000006,0.000026,9.999682e-01
4282,pyopresabsSTCC_qual,CFBRSa28,0,0,0.999288,0.000176,5.361527e-04


In [51]:
y_prob4 = df_proba4[df_proba4['phage']=='p11presabs_qual'].iloc[:,-3:]
y_prob4 = y_prob4.to_numpy()
y_prob4

array([[1.80251380e-06, 1.10806190e-03, 9.98890100e-01],
       [9.99664660e-01, 1.93185710e-04, 1.42152130e-04],
       [9.99228500e-01, 6.81630340e-04, 8.99736550e-05],
       [9.88446350e-01, 1.04139400e-02, 1.13964500e-03],
       [9.88446350e-01, 1.04139400e-02, 1.13964500e-03],
       [2.02436730e-02, 9.68955340e-01, 1.08009910e-02],
       [9.46789600e-01, 3.96002680e-02, 1.36100550e-02],
       [4.51099740e-04, 1.28171980e-02, 9.86731700e-01],
       [9.55403400e-01, 4.41104320e-02, 4.86224000e-04],
       [9.98487600e-01, 1.37558350e-03, 1.36718050e-04],
       [2.05551370e-02, 9.35666200e-01, 4.37785950e-02],
       [8.00717950e-01, 1.35962900e-01, 6.33191400e-02],
       [5.90009150e-04, 1.11505700e-02, 9.88259430e-01],
       [9.55403400e-01, 4.41104320e-02, 4.86224000e-04],
       [2.31649680e-05, 2.37071170e-01, 7.62905600e-01],
       [9.95546460e-01, 1.34978780e-04, 4.31862660e-03],
       [9.55403400e-01, 4.41104320e-02, 4.86224000e-04],
       [9.98963500e-01, 8.76140

In [52]:
ovo4 = rocauc_ovo(y_test_over, y_prob4, average="macro", multi_class="ovo")
ovo4

0.9646724636785798

In [53]:
ovr4 = rocauc_ovr(y_test_over, y_prob4, average="macro", multi_class="ovr")
ovr4

0.9646724636785798

In [54]:
ovos = [ovo1, ovo2, ovo3, ovo4]
np.mean(ovos)

0.9676647334713084

In [55]:
np.std(ovos)

0.0036095930435103354

In [56]:
ovrs = [ovr1, ovr2, ovr3, ovr4]
np.mean(ovrs)

0.9676647334713084

In [57]:
np.std(ovrs)

0.0036095930435103354

In [98]:
accs = [acc_test_over, acc_test_over2, acc_test_over3, acc_test_over4]

In [99]:
mean = np.mean(accs)
print('over-sampling test accuracy mean: %.2f%%' % (mean*100))

over-sampling test accuracy mean: 89.72%


In [100]:
std = np.std(accs)
print('over-sampling test accuracy standard deviation:', std)

over-sampling test accuracy standard deviation: 0.015865159093204316


In [101]:
accs_train = [np.mean(hist1_over.history['accuracy']), np.mean(hist1_over2.history['accuracy']), np.mean(hist1_over3.history['accuracy']),
             np.mean(hist1_over4.history['accuracy'])]

In [102]:
mean_train = np.mean(accs_train)
print('over-sampling train accuracy mean: %.2f%%' % (mean_train*100))

over-sampling train accuracy mean: 96.64%


In [103]:
std_train = np.std(accs_train)
print('over-sampling train accuracy standard deviation:', std_train)

over-sampling train accuracy standard deviation: 0.0060931975
